# Python Notes

## Initialize

In [13]:
# %load utils/measure.py
import time
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter_ns()
        result = func(*args, **kwargs)
        end = time.perf_counter_ns()
        elapsed_ns = end - start
        
        if elapsed_ns < 1_000:
            time_str = f"{elapsed_ns} ns"
        elif elapsed_ns < 1_000_000:
            time_str = f"{elapsed_ns / 1_000:.3f} µs"
        elif elapsed_ns < 1_000_000_000:
            time_str = f"{elapsed_ns / 1_000_000:.3f} ms"
        else:
            time_str = f"{elapsed_ns / 1_000_000_000:.3f} s"
        
        print(f"Performance: {func.__name__}: {time_str}")
        return result
    print("measure-new (util) loaded into global scope.")
    return wrapper


In [15]:
# %load utils/load_libs.py
import matplotlib, math
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline
import cProfile, pstats

## Exceptions

| Exception                  | Example that Raises It                                             |
|---------------------------|---------------------------------------------------------------------|
| `ZeroDivisionError`       | `1 / 0`                                                             |
| `TypeError`               | `'a' + 1`                                                           |
| `NameError`               | `print(undefined_var)`                                             |
| `IndexError`              | `[1, 2, 3][5]`                                                      |
| `KeyError`                | `{'a': 1}['b']`                                                     |
| `AttributeError`          | `None.upper()`                                                     |
| `ValueError`              | `int("abc")`                                                       |
| `ImportError`             | `import doesnotexist`                                              |
| `ModuleNotFoundError`     | `import non_existing_module`                                       |
| `FileNotFoundError`       | `open('no_such_file.txt')`                                         |
| `PermissionError`         | `open('/root/secret.txt', 'w')` *(on Unix if not root)*            |
| `IsADirectoryError`       | `open('/')`                                                        |
| `NotADirectoryError`      | `open('file.txt/anotherfile')`                                     |
| `OSError`                 | `os.remove('nonexistent_file.txt')` *(import os first)*            |
| `RuntimeError`            | `raise RuntimeError("Something went wrong")`                       |
| `RecursionError`          | `def f(): f(); f()`                                                |
| `MemoryError`             | `a = 'a' * 10**10` *(depends on system memory)*                    |
| `FloatingPointError`      | `np.seterr(all='raise'); np.divide(1.0, 0.0)` *(needs NumPy)*      |
| `StopIteration`           | `next(iter([]))`                                                   |
| `StopAsyncIteration`      | *(Occurs in async iterators; hard to demo in one line)*            |
| `EOFError`                | `input()` *(then press Ctrl+D/Z)*                                 |
| `IndentationError`        | *(Occurs with bad indentation — can’t be demoed in one line)*      |
| `SyntaxError`             | `eval("def = 5")`                                                  |
| `UnboundLocalError`       | `def f(): print(x); x = 1; f()`                                    |
| `AssertionError`          | `assert False`                                                     |
| `ArithmeticError`         | `raise ArithmeticError("custom arithmetic error")`                 |
| `BufferError`             | `memoryview(bytearray(5)).release(); len(memoryview(bytearray(5)))`|
| `EnvironmentError`        | *(Alias for `OSError` in Python 2 — not typically used in Py3)*     |
| `LookupError`             | `raise LookupError("lookup failed")`                               |
| `OverflowError`           | `math.exp(1000)` *(import math first)*                             |
| `ReferenceError`          | *(Only with weakrefs; complex to show in one-liner)*               |
| `BlockingIOError`         | *(Occurs with non-blocking I/O — advanced use)*                    |
| `BrokenPipeError`         | *(Typically in subprocess communication — system-dependent)*       |
| `ChildProcessError`       | *(Advanced multiprocessing errors — not easy to demo simply)*      |
| `ConnectionError`         | *(Used in network code; subclassed by others below)*               |
| `ConnectionAbortedError`  | *(Networking — raised on aborted connections)*                     |
| `ConnectionRefusedError`  | *(Try connecting to a closed port)*                                |
| `ConnectionResetError`    | *(Remote connection forcibly closed)*                              |
| `FileExistsError`         | `open('existing_file.txt', 'x')` *(if file exists)*                |
| `InterruptedError`        | *(Rare, from system signals or `signal` module)*                   |
| `ProcessLookupError`      | `os.kill(999999, 9)` *(invalid PID)*                               |
| `TimeoutError`            | *(Usually in `asyncio`, `socket`, or file operations)*             |

---

## String Representation of class

In [5]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"({self.x}, {self.y})"

class Point2:
    def __init__(self, x, y):
        self.x = x
        self.y = y

print(Point(10,9))
print(Point2(10,9))
print(Point(2,4).__str__()) # Unnessary, assigning to a string is implicit
str = Point(2,4)
print(str) # (2,4)

(10, 9)
(2, 4)
(2, 4)


## Arithmatic Operators On Classes

In [10]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __add__(self, point):
        return Point (self.x + point.x, self.y + point.y)
        
    def __str__(self):
        return f"({self.x}, {self.y})"

print(Point(10,20) + Point(2,3)) ## (12,23) no.... it was None due to no return

(12, 23)


#### ✅ **Standard Arithmetic Operators**

| Operator | Method             | Example               |
|----------|--------------------|------------------------|
| `+`      | `__add__(self, other)`      | `a + b` |
| `-`      | `__sub__(self, other)`      | `a - b` |
| `*`      | `__mul__(self, other)`      | `a * b` |
| `/`      | `__truediv__(self, other)`  | `a / b` |
| `//`     | `__floordiv__(self, other)` | `a // b` |
| `%`      | `__mod__(self, other)`      | `a % b` |
| `**`     | `__pow__(self, other)`      | `a ** b` |
| `@`      | `__matmul__(self, other)`   | `a @ b` (matrix mult)

---

#### 🔁 **Reflected Operators** (if left operand doesn't support it)

| Operator | Method                 | Example        |
|----------|------------------------|----------------|
| `+`      | `__radd__(self, other)` | `other + self` |
| `-`      | `__rsub__(self, other)` | `other - self` |
| `*`      | `__rmul__(self, other)` | `other * self` |
| `/`      | `__rtruediv__`          | etc.           |
| `//`     | `__rfloordiv__`         |                |
| `%`      | `__rmod__`              |                |
| `**`     | `__rpow__`              |                |
| `@`      | `__rmatmul__`           |                |

---

#### 🔄 **In-place Operators**

| Operator | Method                   | Example   |
|----------|--------------------------|-----------|
| `+=`     | `__iadd__(self, other)`  | `a += b`  |
| `-=`     | `__isub__`               | etc.      |
| `*=`     | `__imul__`               |           |
| `/=`     | `__itruediv__`           |           |
| `//=`    | `__ifloordiv__`          |           |
| `%=`     | `__imod__`               |           |
| `**=`    | `__ipow__`               |           |
| `@=`     | `__imatmul__`            |           |

---

## Performance

In [21]:
@measure
def iterate(n):
    last = 0
    for i in range(1,n+1):
        last = math.sqrt(i)
    print(i)


iterate(100_000_000)

measure-new (util) loaded into global scope.
100000000
Performance: iterate: 6.909 s


In this case, we did 100 million operations which took 6.909 seconds. which is $1.4473874 \times {10}^7$ operations per second, which is what Prof. Madhav Mukund indicated